In [86]:
%load_ext autoreload
%autoreload 2
from storytelling import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
# select en events:
events = pd.read_csv("../dat/tweeterNews/events.csv.gz", header=None,
                     names=["Num","Date","TotalKeyWords","TotalTweets","Keywoards"])

In [85]:
events.sort_values("TotalTweets", ascending=False).head(10)

,Num,Date,TotalKeyWords,TotalTweets,Keywoards
4498,4499,2014-05-28,9,510920,angelou;pakistan;life;woman;maya;risk;dies;dea...
4883,4884,2014-06-12,22,452534,turtle;sao;win;national;follow;home;brazil;pol...
231,232,2013-09-10,32,423756,control;says;apple;december;india;four;gang;fo...
1965,1966,2014-02-02,10,338252,fans;bowl;see;sunday;blog;game;security;xlviii...
1977,1978,2014-02-03,16,313030,broncos;super;title;may;win;million;bowl;blog;...
1257,1258,2013-12-31,25,308703,celebrations;photo;globe;tonight;australia;yea...
1860,1861,2014-01-29,18,301448,boy;grimm;end;unemployment;veto;rep;reporter;y...
1737,1738,2014-01-23,13,281559,prime;ukraine;supreme;resisting;police;justin;...
1688,1689,2014-01-20,2,276219,king;luther
2701,2702,2014-03-08,35,271306,spotted;hairline;photo;slicks;airlines;malaysi...


In [ ]:
events.loc[4883, "Keywords"]

In [25]:
tweets = pd.read_csv("../dat/tweeterNews/tweets.csv.gz", names=["tid", "eid"])

In [26]:
tweets.head()

,tid,eid
0,371043469185540096,1
1,371043472397987841,1
2,371043478404222976,1
3,371043500814385152,1
4,371043512642719744,1


In [27]:
wildfire_yosemite = tweets[tweets["eid"]==5]

In [29]:
wildfire_yosemite.head()

,tid,eid
27927,370581177494941696,5
27928,370581180321910784,5
27929,370581197103300608,5
27930,370581385456545792,5
27931,370581492789153792,5


In [76]:
tids = list(wildfire_yosemite["tid"].values)
len(tids)

12488

In [73]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

In [74]:
writer = csv.writer(open("dat/wildfire_yosemite.csv", "wt"))
header = ["id", "text", "created_at", "retweet_count", "user_id", "user_followers_count",
          "rt_id", "rt_text", "rt_created_at", "rt_retweet_count", "rt_user_id", "rt_user_followers_count"]
writer.writerow(header)

143

In [77]:
i = 0; j = 100
while 1:
    tweet = api.statuses_lookup(tids[i:j])
    Tweets2CSV(tweet, writer)
    i += 100;
    j += 100;
    if j > len(tids):
        break
    if j >= len(tids):
        j = tids

In [88]:
tids.index(477150935882297344)

89933

In [89]:
len(tids)

452534

In [90]:
# 4884
event_id = 4884
event_tweets = tweets[tweets["eid"]==4884]
tids = list(event_tweets["tid"].values)
RetriveDataset(tids[89933:], api, "4884.csv")

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [ ]:
# 4499
event_id = 4499
event_tweets = tweets[tweets["eid"]==event_id]
tids = list(event_tweets["tid"].values)
RetriveDataset(tids, api, "{}.csv".format(event_id))

In [ ]:
# 232
event_id = 232
event_tweets = tweets[tweets["eid"]==event_id]
tids = list(event_tweets["tid"].values)
RetriveDataset(tids, api, "{}.csv".format(event_id))